In [ ]:
import os
import bs4
import json
import time
import lxml
import requests
import cloudscraper
from random import randint
from fake_useragent import UserAgent
from IPython.display import clear_output as clear

In [ ]:
language = 'french'
ua = UserAgent()

In [ ]:
top_dir = os.path.abspath('').replace('corrections', '')
conjugations_data_dir = os.path.join(top_dir, 'conjugations', 'data', 'language-specific')

In [ ]:
headers = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "accept-encoding": "gzip, deflate, br",
           "accept-language": "en-GB,en;q=0.9,es-ES;q=0.8,es;q=0.7,en-US;q=0.6,eu;q=0.5",
           "cache-control": "max-age=0",
           "cookie": "t=238707487; _ga=GA1.2.1376835774.1641262578; _gid=GA1.2.1482423077.1641262578; _fbp=fb.1.1641262579526.851471446",
           "referer": "https://hidemy.name/en/proxy-list/?start=64",
           "sec-ch-ua-mobile": "?0",
           "sec-ch-ua-platform": "macOS",
           "sec-fetch-dest": "document",
           "sec-fetch-mode": "navigate",
           "sec-fetch-site": "same-origin",
           "sec-fetch-user": "?1",
           "upgrade-insecure-requests": "1",
           "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

In [ ]:
def getSpanishDictTranslations(verb):
    output = []

    request = requests.get(f"https://www.spanishdict.com/translate/{verb}")
    soup = bs4.BeautifulSoup(request.text, "lxml")

    if not soup.select('._25QSB23Y'):

        for i in range(0, len(soup.select("._2vd6M2gR"))):
            entry = soup.select("._2vd6M2gR")[i].getText()

            if entry[0:3] == "to " and entry[3:] not in output:
                if entry[3:].replace(' ','').isalpha():
                    output.append(entry[3:])

        for i in range(0, len(soup.select(".gram_cat"))):
            block = soup.select(".gram_cat")[i]

            if verb + 'se' not in block.getText():

                for i in range(0, len(block.select(".tran_main"))):
                    entry = block.select(".tran_main")[i].getText()

                    if entry[0:3] == "to " and entry[3:] not in output:
                        if entry[3:].replace(' ','').isalpha():
                            output.append(entry[3:])
        
    return output

In [ ]:
with open(os.path.join(conjugations_data_dir, f'conjugations_{language}.json'), 'r', encoding = 'utf8') as file:
    conjugations = json.loads(file.read())

In [ ]:
data = {}

In [ ]:
current = 1

In [ ]:
for verb in conjugations:

    if conjugations[verb]['rank'] == current:

        retries = 1

        while True:
            try:
                url = f'https://www.collinsdictionary.com/dictionary/{language}-english/{verb}'
                scraper = cloudscraper.create_scraper(
                    browser={
                        'browser': 'firefox',
                        'platform': 'windows',
                        'mobile': False
                    })
                soup = bs4.BeautifulSoup(scraper.get(url, headers = {"useragent": f"{ua.random}"}).text, 'lxml')

                if 'Cloudflare' not in soup.select('title')[0].getText():
                    translations = []

                    for section in soup.select('.page .dictionary .hom'):
                        if section.select('.hi.rend-sc.pos'):
                            section_type = section.select('.hi.rend-sc.pos')[0]
                            
                            if 'verb' in section_type.getText() and 'adverb' not in section_type.getText():
                                for entry in section.select('.sense:not(.type-example)>.cit.type-translation .quote'):
                                    translation = entry.getText()
                                    
                                    if translation[:3] == 'to ':
                                        translation = translation[3:]
                                    
                                    if translation.replace(' ','').isalpha() and translation not in translations:
                                        translations.append(translation)

                    data[verb] = translations

                    clear(); print(f'{verb} ({conjugations[verb]["rank"]}) - complete ({len(translations)})')
                    break

                else:
                    clear(); print(f'{verb} ({conjugations[verb]["rank"]}) - retrying ({retries})')
                    retries += 1

            except Exception as ex:
                data[verb] = []
                clear(); print(f'{verb} ({conjugations[verb]["rank"]}) - error ({ex})')
                break

        current += 1

        time.sleep(randint(10000, 20000) / 1000)

In [ ]:
with open(os.path.join(top_dir, 'corrections', 'out', f'translations_{language}.json'), 'w', encoding = 'utf8') as file:
    json.dump(data, file, indent = 4, ensure_ascii = False)

In [ ]:
if language == 'spanish':

    with open(os.path.join(top_dir, 'corrections', 'out', f'translations_spanish.json'), 'r', encoding = 'utf8') as file:
        data = json.loads(file.read())

    combined = {}

    for verb in data:
        spanishdict = getSpanishDictTranslations(verb)
        collins = data[verb][::-1]

        output = spanishdict

        for n in range(0, len(collins)):
            
            if collins[n] not in output:
                iterate = True
                x = n

                while iterate:
                    if collins[x] in output:
                        for i in range(0, len(output)):
                            if output[i] == collins[x]:
                                output.insert(i + 1, collins[n])
                                iterate = False
                                break
                    else:
                        if x == 0:
                            iterate = False
                        else:
                            x -= 1
            
            if collins[n] not in output:
                output.append(collins[n])

        combined[verb] = output
        
        clear(); print(f'{verb} ({conjugations[verb]["rank"]}) - complete ({len(collins)} -> {len(output)})')

    with open(os.path.join(top_dir, 'corrections', 'out', f'translations_{language}.json'), 'w', encoding = 'utf8') as file:
        json.dump(combined, file, indent = 4, ensure_ascii = False)